<a href="https://colab.research.google.com/github/afraAntara/590B/blob/main/Normal_NN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

def ode_solution(timestep, t_init, t_end):
    a = 0.5  # Example value for k
    # Define the range of 'c' from 1 to 10
    c_values = range(1, 100)

    def f(y, a):
      return -a * y


    # Define the ODE function
    def ode_function(y, a, c):
        return f(y, a) + c

    # Define the Runge-Kutta 4 method step
    def rk4_step(f, t, y, h, a, c):
        k1 = f(y, a, c)
        k2 = f(y + 0.5 * k1, a, c)
        k3 = f(y + 0.5 * k2, a, c)
        k4 = f(y + k3, a, c)
        return y +((k1 + 2 * k2 + 2 * k3 + k4)*(h/6))

    # Parameters for solving the ODE
    t0 = t_init
    y0 = 0  # Initial condition
    h = timestep  # Step size
    # Initialize arrays for t values and y values
    t_values = np.arange(t0, t_end, h)
    y_values = [y0]
    c = random.choice(list(c_values))
    c_val = [c]
    a_val = [a] * len(t_values)

    # Solving the ODE using RK4 method
    for t in t_values[:-1]:
        c_val.append(c)
        if round(t) % 10 == 0:
            # Select a random value from c_values
            c = random.choice(list(c_values))
        y_values.append(rk4_step(ode_function, t, y_values[-1], h, a, c))

    data = np.column_stack((t_values, y_values, a_val, c_val))

    return data


def plot(data):
    t = data[:,0]
    y = data[:,1]
    # Plotting the results
    plt.figure(figsize=(12, 6))
    plt.plot(t, y)
    plt.title('Solution of the ODE')
    plt.xlabel('Timesteps')
    plt.ylabel('x(t)')
    plt.grid(True)
    plt.show()


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split

class RungeKuttaNNPyTorch(nn.Module):
    def __init__(self, timestep, t_init, t_end, n_var, n_out, n_oppar, n_hidden1=10):
        super(RungeKuttaNNPyTorch, self).__init__()
        self.timestep = timestep
        self.t_init = t_init
        self.t_end = t_end
        self.n_var = n_var
        self.n_out = n_out
        self.n_oppar = n_oppar
        self.n_inputs = n_var + n_oppar

        # Define the layers
        self.layer_1 = nn.Linear(self.n_inputs, n_hidden1)
        # self.layer_2 = nn.Linear(n_hidden1, n_hidden2)
        self.output_layer = nn.Linear(n_hidden1, n_out)

    def forward(self, y, a, c, x):
        # Forward pass through the network
        stack = torch.cat([y, a, c], dim=1)
        layer_1_out = torch.sigmoid(self.layer_1(stack))
        # layer_2_out = torch.relu(self.layer_2(layer_1_out))
        out = self.output_layer(layer_1_out)
        return out


# Instantiate the model with arbitrary parameters
model = RungeKuttaNNPyTorch(timestep=0.01, t_init=0, t_end=10, n_var=2, n_out=1, n_oppar=1)

# Print model structure
print(model)